In [4]:
# Full modeling pipeline (fixed stem call)
import os
from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.regression.quantile_regression import QuantReg

warnings.filterwarnings("ignore")
sns.set(style="whitegrid", font_scale=1.05)

# -------------------------
# Config: path to cleaned CSV
# -------------------------
CSV_PATH = Path("./Output/merged_clean_panel.csv")  # <-- change if your CSV path differs
OUT_DIR = Path("./Output/model_output")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# -------------------------
# Load data
# -------------------------
if not CSV_PATH.exists():
    raise FileNotFoundError(f"Cleaned CSV not found at {CSV_PATH}. Please save merged_clean_panel.csv at this path and re-run.")
df = pd.read_csv(CSV_PATH)

# Ensure numerics
for c in ['HDI_2023','HDI_sq','GDP_per_capita','log_GDP_per_capita','Suicide_rate']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# -------------------------
# Prepare model dataframe
# -------------------------
df_model = df.dropna(subset=['Suicide_rate','HDI_2023','HDI_sq']).copy()
n_obs = len(df_model)

df_model['high_data_quality'] = df_model.get('Low_data_quality_flag', '').astype(str).str.contains('Sufficient', na=False)

# -------------------------
# Utility functions
# -------------------------
def save_text(txt, filename):
    p = OUT_DIR / filename
    with open(p, "w", encoding="utf-8") as f:
        f.write(txt)
    return p

def compute_vif(df_, features):
    X = df_[features].dropna()
    X = sm.add_constant(X)
    vif_rows = []
    for i, var in enumerate(X.columns):
        if var == 'const':
            continue
        try:
            vif = variance_inflation_factor(X.values, i)
        except Exception:
            vif = np.nan
        vif_rows.append((var, float(vif)))
    return pd.DataFrame(vif_rows, columns=['variable','VIF'])

# -------------------------
# 1) Models
# -------------------------
formula_base = "Suicide_rate ~ HDI_2023"
res_base = smf.ols(formula=formula_base, data=df_model).fit(cov_type='HC1')

if 'log_GDP_per_capita' not in df_model.columns:
    if 'GDP_per_capita' in df_model.columns:
        df_model['log_GDP_per_capita'] = np.log(df_model['GDP_per_capita'].where(df_model['GDP_per_capita']>0, np.nan))
    else:
        df_model['log_GDP_per_capita'] = np.nan

formula_quad = "Suicide_rate ~ HDI_2023 + HDI_sq + log_GDP_per_capita"
res_quad = smf.ols(formula=formula_quad, data=df_model).fit(cov_type='HC1')

# Region fixed effects + income group
res_region = None
if 'continent' in df_model.columns or 'income_group_auto' in df_model.columns:
    df_region = df_model.copy()
    df_region = df_region.dropna(subset=['continent']) if 'continent' in df_region.columns else df_region
    add_controls = []
    if 'continent' in df_region.columns:
        add_controls.append("C(continent)")
    if 'income_group_auto' in df_region.columns:
        add_controls.append("C(income_group_auto)")
    controls = " + ".join(add_controls)
    formula_region = "Suicide_rate ~ HDI_2023 + HDI_sq + log_GDP_per_capita" + (" + " + controls if controls else "")
    res_region = smf.ols(formula=formula_region, data=df_region).fit(cov_type='HC1')

df_high = df_model[df_model['high_data_quality']==True].copy()
res_high = smf.ols(formula=formula_quad, data=df_high).fit(cov_type='HC1')

# Quantile regression
try:
    X_q = sm.add_constant(df_model[['HDI_2023','HDI_sq','log_GDP_per_capita']].fillna(0))
    y_q = df_model['Suicide_rate']
    qr = QuantReg(y_q, X_q)
    res_qr = qr.fit(q=0.5)
except Exception:
    res_qr = None

# Save model summaries
save_text(res_base.summary().as_text(), "01_model_base_summary.txt")
save_text(res_quad.summary().as_text(), "02_model_quad_summary.txt")
if res_region is not None:
    save_text(res_region.summary().as_text(), "03_model_region_summary.txt")
save_text(res_high.summary().as_text(), "04_model_high_quality_summary.txt")
if res_qr is not None:
    save_text(res_qr.summary().as_text(), "05_model_quantile_median_summary.txt")

# -------------------------
# 2) Diagnostics & plots
# -------------------------
vif_df = compute_vif(df_model, ['HDI_2023','HDI_sq','log_GDP_per_capita'])
vif_df.to_csv(OUT_DIR / "vif_quad.csv", index=False)

bp = het_breuschpagan(res_quad.resid, res_quad.model.exog)
bp_txt = f"Breusch-Pagan LM stat: {bp[0]:.4f}, p-value: {bp[1]:.4g}, fvalue: {bp[2]:.4f}, f pvalue: {bp[3]:.4g}"
save_text(bp_txt, "breusch_pagan_quad.txt")

influence = res_quad.get_influence()
cooks_d = influence.cooks_distance[0]
df_model['cooks_d'] = cooks_d
df_model[['Country Name','cooks_d']].sort_values('cooks_d', ascending=False).head(20).to_csv(OUT_DIR / "top_cooks.csv", index=False)

# Fixed: use stem without unsupported kwarg
plt.figure(figsize=(10,4))
plt.stem(np.arange(len(cooks_d)), cooks_d, markerfmt=",")
plt.xlabel("Observation index")
plt.ylabel("Cook's distance")
plt.title("Cook's distance (quad model)")
plt.tight_layout()
plt.savefig(OUT_DIR / "cooks_distance.png", dpi=150)
plt.close()

# Residuals vs Fitted
fitted = res_quad.fittedvalues
resid = res_quad.resid
plt.figure(figsize=(8,5))
plt.scatter(fitted, resid, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (Quadratic model)")
plt.tight_layout()
plt.savefig(OUT_DIR / "residuals_vs_fitted_quad.png", dpi=150)
plt.close()

# QQ-plot
sm.qqplot(resid, line='s')
plt.title("QQ-plot residuals (Quadratic model)")
plt.tight_layout()
plt.savefig(OUT_DIR / "qqplot_resid_quad.png", dpi=150)
plt.close()

# Partial effect plot
hdix = np.linspace(df_model['HDI_2023'].min(), df_model['HDI_2023'].max(), 200)
mean_loggdp = df_model['log_GDP_per_capita'].mean()
p = res_quad.params
yhat = p.get('Intercept',0) + p.get('HDI_2023',0)*hdix + p.get('HDI_sq',0)*(hdix**2) + p.get('log_GDP_per_capita',0)*mean_loggdp
plt.figure(figsize=(8,6))
plt.scatter(df_model['HDI_2023'], df_model['Suicide_rate'], alpha=0.5, label='observed')
plt.plot(hdix, yhat, color='red', linewidth=2, label='predicted (quad)')
tipping = None
if ('HDI_2023' in p) and ('HDI_sq' in p) and p['HDI_sq']!=0:
    tipping = -p['HDI_2023']/(2*p['HDI_sq'])
    if np.isfinite(tipping):
        plt.axvline(tipping, color='blue', linestyle='--', label=f'tipping ≈ {tipping:.3f}')
plt.xlabel("HDI_2023")
plt.ylabel("Suicide_rate")
plt.title("Predicted Suicide Rate by HDI (Quadratic model)")
plt.legend()
plt.tight_layout()
plt.savefig(OUT_DIR / "predicted_hdi_quad.png", dpi=150)
plt.close()

vif_df.to_csv(OUT_DIR / "vif_for_display.csv", index=False)

# -------------------------
# 3) Bootstrap tipping (optional)
# -------------------------
def bootstrap_tipping(df_in, n_boot=500, seed=1234):
    np.random.seed(seed)
    tpoints = []
    data = df_in[['Suicide_rate','HDI_2023','HDI_sq','log_GDP_per_capita']].dropna()
    n = len(data)
    if n < 30:
        return None
    for i in range(n_boot):
        samp = data.sample(n, replace=True)
        try:
            modd = smf.ols("Suicide_rate ~ HDI_2023 + HDI_sq + log_GDP_per_capita", data=samp).fit()
            b1 = modd.params.get('HDI_2023', np.nan)
            b2 = modd.params.get('HDI_sq', np.nan)
            if b2 != 0 and np.isfinite(b1) and np.isfinite(b2):
                tpoints.append(-b1/(2*b2))
        except Exception:
            continue
    if len(tpoints)==0:
        return None
    arr = np.array(tpoints)
    return {'n': len(arr), 'median': np.nanmedian(arr), '2.5%': np.nanpercentile(arr,2.5), '97.5%': np.nanpercentile(arr,97.5)}

bs = bootstrap_tipping(df_model, n_boot=500)
if bs is not None:
    save_text(str(bs), "bootstrap_tipping_point.txt")

# -------------------------
# 4) Robustness: drop top influencers and refit
# -------------------------
top_infl = df_model.sort_values('cooks_d', ascending=False).head(2).index
df_noinfl = df_model.drop(index=top_infl)
res_quad_noinfl = smf.ols(formula_quad, data=df_noinfl).fit(cov_type='HC1')
save_text(res_quad_noinfl.summary().as_text(), "model_quad_no_influentials.txt")

# -------------------------
# 5) Export overview report
# -------------------------
lines = []
lines.append("MODEL REPORT - The Price of Progress\n")
lines.append(f"Data file: {CSV_PATH}\nObservations used in quad model: {n_obs}\n")
lines.append("=== Quadratic Model (HDI + HDI^2 + log GDP) summary (robust SE) ===\n")
lines.append(res_quad.summary().as_text())
lines.append("\n=== VIF (quad predictors) ===\n")
lines.append(vif_df.to_string(index=False))
lines.append("\n=== Breusch-Pagan test (quad) ===\n")
lines.append(bp_txt)
if bs is not None:
    lines.append("\n=== Bootstrap tipping point CI ===\n")
    lines.append(str(bs))
lines.append("\n=== Top influencers (Cook's distance) ===\n")
lines.append(df_model[['Country Name','cooks_d']].sort_values('cooks_d', ascending=False).head(10).to_string(index=False))

save_text("\n\n".join(lines), "final_model_report.txt")

# -------------------------
# Print concise on-screen summary
# -------------------------
print("Modeling complete. Outputs saved to:", OUT_DIR)
print("Quadratic model coefficients (robust SE):")
print(res_quad.summary().tables[1])

if tipping is not None:
    print(f"\nEstimated tipping point (HDI*): {tipping:.3f}")
if bs is not None:
    print("Bootstrap tipping point CI saved (see bootstrap_tipping_point.txt) and shows:", bs)

print("\nFiles saved (sample):")
for p in sorted(OUT_DIR.iterdir()):
    print(" -", p.name)


Modeling complete. Outputs saved to: Output\model_output
Quadratic model coefficients (robust SE):
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             21.1069      9.184      2.298      0.022       3.106      39.107
HDI_2023             -41.4026     21.420     -1.933      0.053     -83.386       0.580
HDI_sq                37.8075     17.471      2.164      0.030       3.565      72.050
log_GDP_per_capita    -0.3835      0.725     -0.529      0.597      -1.804       1.037

Estimated tipping point (HDI*): 0.548
Bootstrap tipping point CI saved (see bootstrap_tipping_point.txt) and shows: {'n': 500, 'median': np.float64(0.5455938997879175), '2.5%': np.float64(0.17750119011523557), '97.5%': np.float64(0.7060643850959668)}

Files saved (sample):
 - 01_model_base_summary.txt
 - 02_model_quad_summary.txt
 - 03_model_region_summary.txt
 - 04_model_hig